<a href="https://colab.research.google.com/github/LikhanInSpace/bomFatabo/blob/main/vgg16_model_for_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from __future__ import print_function
!pip install keras_applications

import numpy as np
import warnings
from keras.models import Model
from keras.layers import  Flatten, Dense, Dropout
from keras.layers import  Input, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs

In [41]:
def VGG16(input_tensor=None, classes=2):
  img_rows, img_cols= 224,224
  img_channels= 3

  img_dim=(img_rows,img_cols,img_channels)
  img_input = Input(shape=img_dim)

  # block 1
  x = Convolution2D(64, (3,3), activation='relu', padding='same', name='block1_conv1')(img_input)
  x = Convolution2D(64, (3,3), activation='relu', padding='same', name='block1_conv2')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(x)

  # block 2
  x = Convolution2D(128, (3,3), activation='relu', padding='same', name='block2_conv1')(x)
  x = Convolution2D(128, (3,3), activation='relu', padding='same', name='block2_conv2')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block2_pool')(x)

  # block 3
  x = Convolution2D(256, (3,3), activation='relu', padding='same', name='block3_conv1')(x)
  x = Convolution2D(256, (3,3), activation='relu', padding='same', name='block3_conv2')(x)
  x = Convolution2D(256, (3,3), activation='relu', padding='same', name='block3_conv3')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block3_pool')(x)

  # block 4
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block4_conv1')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block4_conv2')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block4_conv3')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block4_pool')(x)

  # block 5
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block5_conv1')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block5_conv2')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block5_conv3')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block5_pool')(x)

  # classification block
  x = Flatten(name='flatten')(x)
  x = Dense(4096, activation='relu', name= 'fc1')(x)
  x = Dense(4096, activation='relu', name= 'fc2')(x)
  x = Dense(classes, activation='softmax', name='predictions')(x)

  # create model

  model = Model(inputs = img_input, outputs = x, name='vgg16')
  return model






In [44]:
#new section

model = VGG16(classes = 2) # for real and fake image
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
dataset_path = os.listdir('/content/drive/MyDrive/thesis dataset/type of image')
image_types = os.listdir('/content/drive/MyDrive/thesis dataset/type of image')
print (image_types) # what kinds image in this dataset
print("Types of image found:", len(dataset_path))

['so called fake', 'so called real']
Types of image found: 2


In [67]:
images = []
for item in image_types:
  all_images = os.listdir('/content/drive/MyDrive/thesis dataset/type of image' + '/' + item)

for image in all_images:
  images.append((item, str('/content/drive/MyDrive/thesis dataset/type of image'+ '/' + item) + '/' + image))
  print(images)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

